In [ ]:
!conda info --envs

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!ls h5ad_data/

In [ ]:
data = !ls h5ad_data/
data

In [ ]:
file_name = [x[:-5] for x in data]

In [ ]:
data_ad = ["h5ad_data/" + x for x in data]

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvi

In [ ]:
for x in range(len(data)):
    adata = sc.read_h5ad(data_ad[x])
    sc.pp.filter_genes(adata, min_cells = 10)
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    #modified _model.py of solo referring to https://discourse.scverse.org/t/solo-from-scvi-model-throwing-attributeerror/1380
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df["prediction"] = solo.predict(soft = False)
    doublets = df[(df.prediction == 'doublet')]
    
    adata = sc.read_h5ad(data_ad[x])
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]
    
    #Basic filtering
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)
    
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    
    adata = adata[adata.obs.pct_counts_mt < 20, :]
    
    adata.write_h5ad(f"/Users/titai/Documents/Research_Projects/BIG-TCR/03_scRNA_integration_pseudo_time_analysis/preprocessed_h5ad_data/{file_name[x]}_preprocessed.h5ad")